In [19]:
# main_script.py
import random
import time
import logging
import csv
import json
import requests
import csv
import random
from pathlib import Path

import json
from typing import Dict, List

# configs.py

# Workable integration token (token)
token = '4pA5n-Ci0iA4o8u_DesrZpzkQxFxBDKkQ9id5klwmqw'

base_url = 'https://assignmentcorporate.workable.com/spi/v3/'

headers = {
    'accept': 'application/json',
    'content-type': 'application/json',
    'Authorization': f'Bearer {token}'
    }

class JobsDTO:
    def __init__(self, title: str, shortcode: str):
        self.title = title
        self.shortcode = shortcode
        
    def __str__(self) -> str:
        return f"Title: {self.title}, Shortcode: {self.shortcode}"

class CandidateDTO:
    def __init__(self, name: str, email: str, phone: str, address: str, job: Dict, domain: str, disqualified: bool, sourced: bool) -> None:
        self.name = name
        self.email = email
        self.phone = phone
        self.address = address
        self.job = job
        self.domain = domain
        self.disqualified = disqualified
        self.sourced = sourced # To prevent email notifications
    
    # Getter method for job
    def get_job(self) -> Dict:
        return self.job

    # Getter method for shortcode
    def get_shortcode(self) -> str:
        if 'shortcode' in self.job:
            return self.job['shortcode']
        else:
            return None
        
    def to_json(self) -> str:
        """
        Convert the CandidateDTO object to JSON format.
        """
        dto_dict = {
            "name": self.name,
            "email": self.email,
            "phone": self.phone,
            "address": self.address,
            "job": {
                "shortcode": self.job['shortcode'],
                "title": self.job['title']
            },
            "domain": self.domain,
            "disqualified": self.disqualified,
            "sourced": self.sourced
        }
        return json.dumps(dto_dict, indent=4)

import json

class CandidateResponseDTO:
    def __init__(self, response):
        # Extract candidate data from the response
        candidate_data = response.get('candidate', {})
        self.id = candidate_data.get('id')
        self.name = candidate_data.get('name')
        self.firstname = candidate_data.get('firstname')
        self.lastname = candidate_data.get('lastname')
        self.headline = candidate_data.get('headline')
        self.image_url = candidate_data.get('image_url')
        self.account = candidate_data.get('account', {})
        self.job = candidate_data.get('job', {})
        self.stage = candidate_data.get('stage')
        self.disqualified = candidate_data.get('disqualified')
        self.disqualified_at = candidate_data.get('disqualified_at')
        self.disqualification_reason = candidate_data.get('disqualification_reason')
        self.hired_at = candidate_data.get('hired_at')
        self.sourced = candidate_data.get('sourced')
        self.profile_url = candidate_data.get('profile_url')
        self.address = candidate_data.get('address')
        self.phone = candidate_data.get('phone')
        self.email = candidate_data.get('email')
        self.outbound_mailbox = candidate_data.get('outbound_mailbox')
        self.domain = candidate_data.get('domain')
        self.uploader_id = candidate_data.get('uploader_id')
        self.created_at = candidate_data.get('created_at')
        self.updated_at = candidate_data.get('updated_at')
        self.cover_letter = candidate_data.get('cover_letter')
        self.summary = candidate_data.get('summary')
        self.education_entries = candidate_data.get('education_entries', [])
        self.experience_entries = candidate_data.get('experience_entries', [])
        self.skills = candidate_data.get('skills', [])
        self.answers = candidate_data.get('answers', [])
        self.resume_url = candidate_data.get('resume_url')
        self.tags = candidate_data.get('tags', [])
        self.location = candidate_data.get('location', {})
        self.originating_candidate_id = candidate_data.get('originating_candidate_id')

        # Additional attributes for the talent pool type of candidate
        self.talent_pool = candidate_data.get('talent_pool', {})

    def to_json(self):
        # Convert the attributes to a JSON-compatible dictionary
        json_data = {
            'id': self.id,
            'name': self.name,
            'firstname': self.firstname,
            'lastname': self.lastname,
            'headline': self.headline,
            'image_url': self.image_url,
            'account': self.account,
            'job': self.job,
            'stage': self.stage,
            'disqualified': self.disqualified,
            'disqualified_at': self.disqualified_at,
            'disqualification_reason': self.disqualification_reason,
            'hired_at': self.hired_at,
            'sourced': self.sourced,
            'profile_url': self.profile_url,
            'address': self.address,
            'phone': self.phone,
            'email': self.email,
            'outbound_mailbox': self.outbound_mailbox,
            'domain': self.domain,
            'uploader_id': self.uploader_id,
            'created_at': self.created_at,
            'updated_at': self.updated_at,
            'cover_letter': self.cover_letter,
            'summary': self.summary,
            'education_entries': self.education_entries,
            'experience_entries': self.experience_entries,
            'skills': self.skills,
            'answers': self.answers,
            'resume_url': self.resume_url,
            'tags': self.tags,
            'location': self.location,
            'originating_candidate_id': self.originating_candidate_id,
            'talent_pool': self.talent_pool  # Include talent_pool attribute if present
        }
        return json.dumps(json_data, indent=4)

    @classmethod
    def from_json(cls, data):
        # Create an instance of CandidateResponseDTO using the response data
        return cls(data)


    # @classmethod
    # def from_json(cls, data):
    #     candidate_data = data.get('candidate', {})
    #     return cls(
    #         id=candidate_data.get('id'),
    #         name=candidate_data.get('name'),
    #         firstname=candidate_data.get('firstname'),
    #         lastname=candidate_data.get('lastname'),
    #         headline=candidate_data.get('headline'),
    #         image_url=candidate_data.get('image_url'),
    #         account=candidate_data.get('account'),
    #         job=candidate_data.get('job'),
    #         stage=candidate_data.get('stage'),
    #         disqualified=candidate_data.get('disqualified'),
    #         disqualified_at=candidate_data.get('disqualified_at'),
    #         disqualification_reason=candidate_data.get('disqualification_reason'),
    #         hired_at=candidate_data.get('hired_at'),
    #         sourced=candidate_data.get('sourced'),
    #         profile_url=candidate_data.get('profile_url'),
    #         address=candidate_data.get('address'),
    #         phone=candidate_data.get('phone'),
    #         email=candidate_data.get('email'),
    #         outbound_mailbox=candidate_data.get('outbound_mailbox'),
    #         domain=candidate_data.get('domain'),
    #         uploader_id=candidate_data.get('uploader_id'),
    #         created_at=candidate_data.get('created_at'),
    #         updated_at=candidate_data.get('updated_at'),
    #         cover_letter=candidate_data.get('cover_letter'),
    #         summary=candidate_data.get('summary'),
    #         education_entries=candidate_data.get('education_entries'),
    #         experience_entries=candidate_data.get('experience_entries'),
    #         skills=candidate_data.get('skills'),
    #         answers=candidate_data.get('answers'),
    #         resume_url=candidate_data.get('resume_url'),
    #         tags=candidate_data.get('tags'),
    #         location=candidate_data.get('location'),
    #         originating_candidate_id=candidate_data.get('originating_candidate_id')
    #     )

# logging/logging_config.py

def configure_logging(log_file_path):
    try:
        logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    except FileNotFoundError as e:
        # Log the error to a separate error log file
        error_logger = logging.getLogger('error_logger')
        error_logger.error(f"Error setting up logging: Log file path '{log_file_path}' not found: {e}")
    except PermissionError as e:
        # Log the error to a separate error log file
        error_logger = logging.getLogger('error_logger')
        error_logger.error(f"Error setting up logging: Permission denied for '{log_file_path}': {e}")
    except Exception as e:
        # Log the error to a separate error log file
        error_logger = logging.getLogger('error_logger')
        error_logger.exception("Error setting up logging:")
        # Print a generic error message
        print("An error occurred while setting up logging. Please check the log files for details.")


# initialize.py
import os
import time

'''
This code initializes logging for the project by creating a timestamped log file.
It first ensures the existence of a directory for log files.
Then, it generates a current timestamp using the time.strftime() function and appends it to the log file name.
Each log file created has a unique name based on the timestamp when the project is run.
Finally, it configures logging using the provided log file path.
'''
# Set up logging with a timestamped log file
log_directory = 'project/logs'
if not os.path.exists(log_directory):
    os.makedirs(log_directory)

current_time = time.strftime("%Y-%m-%d_%H-%M-%S")
log_file_path = os.path.join(log_directory, f'customer_{current_time}.log')  # Unique log file path with timestamp
# if path is wrong file will not be created !
configure_logging(log_file_path)







# Read CSV file and return a list of dictionaries
def csv_to_dict_list(file_path):
    try:
        with open(file_path, 'r') as csv_file:
            reader = csv.DictReader(csv_file)
            data = [row for row in reader]
        logging.info('CSV file read successfully.')
        return data
    except Exception as e:
        logging.error(f"Error reading CSV file: {e}")
        return []

# Process each row of data
def process_data(data):
    for row_number, row in enumerate(data, start=1):
        try:
            # Process each row of data here
            logging.info(f"Processed row #{row_number}: {row}")
        except Exception as e:
            logging.error(f"Error processing row #{row_number}: {e}")

def generate_email_suffix(candidate_data):
    first_name = candidate_data['First Name'][0:1].lower()
    last_name = candidate_data['Last Name'][:2].lower()
    email = candidate_data['Email']
    current_time = int(time.time())
    random.seed(current_time)
    random_integer = str(random.randint(100, 999))  # Generate a random integer between 100 and 999
    # Create the email format
    email_format = f"{first_name}{last_name}_{email[:-4]}{random_integer}{email[-4:]}"
    return email_format

def generate_full_address(candidate_data):
    address = candidate_data['Address']
    address_2 = candidate_data['Address 2']
    city = candidate_data['City']
    state = candidate_data['State']
    country = candidate_data['Country']
    zip_code = candidate_data['Zip']
    # Concatenate the address components
    full_address = ', '.join(filter(None, [address, address_2, city, state, country, zip_code]))
    return full_address

def is_disqualified(candidate_data):
    status = candidate_data['Status']
    # Check if the status is 'Rejected'
    disqualified = True if status == 'Rejected' else False
    return disqualified

def position(candidate_data, jobs):
    candidate_position = candidate_data['Position']
    for job in jobs:
        if candidate_position == job.title:
            return {
                "shortcode": job.shortcode,
                "title": job.title
            }
    return None


def import_candidate(candidates_data, headers):
    responses = []
    
    # importing shortcode in the POST url
    url = f'{base_url}jobs/{candidates_data.job["shortcode"]}/candidates'
    
    candidate_json = candidates_data.to_json()
    
    # Set response parameters
    response = requests.post(url, candidate_json, headers=headers)
    
    # Check response status
    if response.status_code == 201:
        print(f'Candidate {candidates_data.name} imported successfully!')
        logging.info(f'Candidate {candidates_data.name} imported successfully!')
        
        
        # Response as JSON
        response_json = json.loads(response.text)
        logging.info(json.dumps(response_json, indent=4))
        print(json.dumps(response_json, indent=4))
        # Convert JSON response to DTO object
        candidate_response = CandidateResponseDTO.from_json(response_json)

        # Print response details
        print(f'Response ID: {candidate_response.id}')
        print(f'Response Name: {candidate_response.name}')
        
        logging.info('Response ID: %s', candidate_response.id)
        logging.info('Response Name: %s', candidate_response.name)
        logging.info('Response: %s', candidate_response.to_json)
        responses.append(candidate_response)
    else:
        logging.error(f'Failed to import candidate {candidates_data.name}.')
        print(f'Failed to import candidate {candidates_data.name}.')
        print('Response:', response.text)
        logging.error('Response: %s', response.text)
        print('Response status:', response.status_code)
        logging.error('Response status: %d', response.status_code)

    return responses


def import_candidate_to_pool(candidates_data, headers):
    responsespool=[]
    candidate = candidates_data
    if candidate.sourced:
        stage = "Applied"
    else:
        stage = "Sourced"
    url = f'{base_url}talent_pool/{stage}/candidates'
    print(url)
    print("Importing candidate ",{candidate.name})
    
    payload = candidate.to_json()
    response = requests.post(url, payload, headers=headers)

    if response.status_code == 201:
        response_json = json.loads(response.text)
        candidate_response = CandidateResponseDTO.from_json(response_json)
        responsespool.append(candidate_response)
    else:
        print(f'Failed to import candidate {candidate.name}.')
        print('Response status:', response.status_code)

    return responsespool



# Get request based on {id}
def get_candidate_info(base_url, id, headers):
    url = f'{base_url}candidates/{id}'  # url = f'{base_url}{shortcode}/candidates/{id}'
    logging.info("URL: %s", url)
    try:
        # Send the GET request
        response = requests.get(url, headers=headers)
        
        # Check the response status code
        if response.status_code == 200:
            logging.info(response.text)
            print("Response:")
            print(response.text)
        else:
            print(f"Error: {response.status_code} - {response.reason}")
            logging.error("Error: %d - %s", response.status_code, response.reason)
    except requests.exceptions.RequestException as e:
        logging.error("Error: %s", e)

search_candidate_by_id = lambda responses, id: next((response.__dict__ for response in responses if response.id == id), None)


# Main function
def main():
    
    logging.info('Script started')

    # Read CSV file and process data
    file_path = Path('customer_import.csv')
    logging.info('Starting data import...')
    print('Starting data import...')

    csv_data = csv_to_dict_list(file_path)

    if csv_data:
        print('CSV file read successfully.')
        logging.info('CSV file read successfully.')
        print('Importing Candidates Information from CSV file...')
        logging.info('Importing Candidates Information from CSV file...')
        process_data(csv_data)
        print('Data import completed.')
        logging.info('Data import completed.')
    else:
        print('Error reading CSV file. Please check the log for details.')
        logging.error('Error reading CSV file. Data import aborted.')

    # Fetch job data from get jobs request to the API
    res = requests.get(base_url + 'jobs', headers=headers)
    job_data = res.json()  # Parse JSON response

    # Check if the response is successful
    if res.status_code == 200:
        jobs = []
        for job in job_data['jobs']:
            # Create JobsDTO object for each job
            job_dto = JobsDTO(
                job['title'],
                job['shortcode'],
            )
            jobs.append(job_dto)

        # # Print the attributes of JobsDTO objects
        # for job in jobs:
        #     print(vars(job))  # Print object attributes as dictionary
    else:
        print("Failed to fetch job data:", res.status_code)

    # # Log the list of jobs
    # logging.info(f"Jobs: {repr([str(job) for job in jobs])}")
    logging.info('Creating job objects from given data')
    for job in jobs:
        logging.info(vars(job))

    # Parse the CSV data and create CandidateDTO objects
    candidates = []
    for candidate_data in csv_data:
        candidate = CandidateDTO(
            candidate_data['First Name'] + ' ' + candidate_data['Last Name'],
            generate_email_suffix(candidate_data),
            candidate_data['Phone'],
            generate_full_address(candidate_data),
            position(candidate_data, jobs),
            candidate_data['Referred By'],
            is_disqualified(candidate_data),
            sourced=False  # Set sourced to False to prevent email notifications THIS MEANS CANDIDATE APPLIED
        )
        candidates.append(candidate)

    logging.info('Creating candidate objects from given data')
    # Print the CandidateDTO objects
    for candidate in candidates: # Print object attributes as dictionary
        logging.info(f"Candidate: {candidate.name}")

    # at this point i have objects of this requested form:

    # {
    #         "name": "John Doe",
    #         "email": "jdo_john.doe@email810.com",
    #         "phone": "123-456-7890",
    #         "address": "123 Main St, Anytown, CA, United States, 12345",
    #         "job": {
    #             "shortcode": "F8D1EA3849",
    #             "title": "Data Architect"
    #         },
    #         "domain": "Employee referral",
    #         "disqualified": false,
    #         "sourced": false
    #     }

    # i can send the objects in JSON form for both post requests
        
    # Prompt the user to select the endpoint
    print("\nSelect an endpoint to send a POST request:")
    print("\n1. /jobs/{shortcode}/candidates")
    print("\n2. /talent_pool/{stage}/candidates")

    choice = input("\nEnter your choice (1 or 2): ")

    responses = []  # Declare and initialize the responses list before using it

    if choice == '1':
        print("\nImporting candidates by POST in Candidates endpoint ...")
        for candidate in candidates:
            responses += import_candidate(candidate, headers)  # Use += to append the responses

    elif choice == '2':
        print("\nImporting candidates by POST in talent_pool endpoint ...")
        for candidate in candidates:
            responses += import_candidate_to_pool(candidate, headers)
    else:
        print("Invalid choice. Please select either 1 or 2.")

    # Prompt the user to perform a GET request
    print("\nDo you want to perform a GET request?")
    print("\n1. Yes")
    print("\n2. No")

    get_choice = input("\nEnter your choice (1 or 2): ")

    if get_choice == '1':
        
        print("\nExecuting GET request...")
        logging.info("Executing GET request...")
        # Iterate through the list of responses
        logging.info(responses)
        for index, response in enumerate(responses, start=1):
            id = response.id
            name = response.name
            try:
                if response.job:
                    job_shortcode = response.job['shortcode'] if response.job else "N/A"
                    print(f"{index}. Name: {name}, ID: {id}, Job Shortcode: {job_shortcode}")
                    logging.info(f"{index}. Name: {name}, ID: {id}, Job Shortcode: {job_shortcode}")
                else:
                    print(f"{index}. Name: {name}, ID: {id}")
                    logging.info(f"{index}. Name: {name}, ID: {id}")
            except Exception as e:
                print(f"Error processing response {index}: {str(e)}")
                logging.error(f"Error processing response {index}: {str(e)}")

        
        # Ask the user to input a number corresponding to a candidate
        selected_number = input("\nEnter the number corresponding to your choice: ")

        # Validate the user input and retrieve the selected candidate details
        try:
            selected_index = int(selected_number)
            if 1 <= selected_index <= len(responses):
                selected_candidate = responses[selected_index - 1]
                selected_id = selected_candidate.id
                # selected_shortcode = selected_candidate.job['shortcode']
                print(f"\nYou selected Candidate {selected_index}. ID: {selected_id}")
                logging.info(f"You selected Candidate {selected_index}. ID: {selected_id}")
            else:
                print("\nInvalid input: Please enter a number within the given range.")
                logging.error("Invalid input: Please enter a number within the given range.")
        except ValueError:
            print("\nInvalid input: Please enter a valid number.")
            logging.error("Invalid input: Please enter a valid number.")

        # id = selected_id
        # shortcode = selected_shortcode

        print("\nChoose an option:")
        print("\n1. Perform a GET request using the API")
        print("\n2. Print candidate data locally")
        choice = input("\nEnter your choice (1 or 2): ")

        if choice == '1':
            get_candidate_info(base_url, selected_id, headers)
        elif choice == '2':
            candidate_data = search_candidate_by_id(responses, selected_id)
            logging.info(candidate_data)
            print(candidate_data)
        else:
            print("Invalid choice. Please enter 1 or 2.")
            logging.error("Invalid choice. Please enter 1 or 2.")

    elif get_choice == '2':
        print("\nNo GET request executed.")
        logging.info("No GET request executed.")
    else:
        print("\nInvalid choice. Please select either 1 or 2.")
        logging.error("Invalid choice. Please select either 1 or 2.")

    input("Press enter to exit...")

if __name__ == "__main__":
    main()

Starting data import...
CSV file read successfully.
Importing Candidates Information from CSV file...
Data import completed.

Select an endpoint to send a POST request:

1. /jobs/{shortcode}/candidates

2. /talent_pool/{stage}/candidates

Importing candidates by POST in talent_pool endpoint ...
https://assignmentcorporate.workable.com/spi/v3/talent_pool/Sourced/candidates
Importing candidate  {'John Doe'}
https://assignmentcorporate.workable.com/spi/v3/talent_pool/Sourced/candidates
Importing candidate  {'Jane Smith'}
https://assignmentcorporate.workable.com/spi/v3/talent_pool/Sourced/candidates
Importing candidate  {'Michael Jones'}
https://assignmentcorporate.workable.com/spi/v3/talent_pool/Sourced/candidates
Importing candidate  {'Sarah Clark'}
https://assignmentcorporate.workable.com/spi/v3/talent_pool/Sourced/candidates
Importing candidate  {'Chris Nguyen'}
https://assignmentcorporate.workable.com/spi/v3/talent_pool/Sourced/candidates
Importing candidate  {'Emily Kim'}
https://ass

In [16]:
for response_data in responses:
    # Create an instance of CandidateResponseDTO using the response data
    response = CandidateResponseDTO.from_json(response_data)
    print(response.to_json())

NameError: name 'responses' is not defined